# 02 - Volatility Models
Demonstrates historical rolling volatility, EWMA, and GARCH(1,1) on daily data. Adjust parameters in the first cell as needed.

In [1]:
import sys
from pathlib import Path

# Ensure repository root is on the path when running from notebooks/
ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))
print(f"Using project root: {ROOT}")

Using project root: /Users/mahdihoumani/Documents/Quant/VolatilityModellingAndOptionPricing/volatility-options-quant


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from src.data_loader import download_price_data
from src.returns import log_returns, TRADING_DAYS
from src.vol_models.historical import rolling_vol
from src.vol_models.ewma import ewma_vol
from src.vol_models.garch import fit_garch11

# Parameters
TICKER = 'SPY'
START = '2018-01-01'
END = None  # set to 'YYYY-MM-DD' to bound the sample
ROLL_WINDOW = 21
EWMA_LAMBDA = 0.94

prices = download_price_data(TICKER, START, END)
returns = log_returns(prices['adj_close']).dropna()
prices.head(), returns.head()

(             adj_close
 Date                  
 2018-01-02  237.208267
 2018-01-03  238.708542
 2018-01-04  239.714706
 2018-01-05  241.312241
 2018-01-08  241.753479,
 Date
 2018-01-03    0.006305
 2018-01-04    0.004206
 2018-01-05    0.006642
 2018-01-08    0.001827
 2018-01-09    0.002261
 Name: log_return, dtype: float64)

In [3]:
# Historical rolling volatility (annualized)
hist_vol = rolling_vol(returns, window=ROLL_WINDOW, annualize=True)
hist_vol.tail()

Date
2025-12-26    0.087272
2025-12-29    0.086215
2025-12-30    0.084681
2025-12-31    0.087334
2026-01-02    0.087330
Name: hist_vol_21, dtype: float64

In [4]:
# EWMA volatility (annualized)
ewma_vol_series = ewma_vol(returns, lam=EWMA_LAMBDA, annualize=True)
ewma_vol_series.tail()

Date
2025-12-26    0.112382
2025-12-29    0.108959
2025-12-30    0.106548
2025-12-31    0.103412
2026-01-02    0.104348
Name: ewma_vol, dtype: float64

In [ ]:
# GARCH(1,1) conditional volatility (annualized)
try:
    garch_res = fit_garch11(returns, annualize=True)
    garch_vol = garch_res['cond_vol']
except ImportError:
    print('arch package not installed; skipping GARCH fit.')
    garch_vol = None
garch_vol.tail() if garch_vol is not None else None

In [ ]:
# Plot the three volatility estimates
plt.figure(figsize=(10, 5))
hist_vol.plot(label=f'Historical {ROLL_WINDOW}d')
ewma_vol_series.plot(label=f'EWMA \u03bb={EWMA_LAMBDA}')
if garch_vol is not None:
    garch_vol.plot(label='GARCH(1,1)')
plt.title(f'Annualized Volatility Estimates for {TICKER}')
plt.ylabel('Volatility')
plt.legend()
plt.tight_layout()
plt.show()